In [1]:
import projectpath

import collections
import os
from typing import Iterable

import pandas as pd

from kb import kb
from scheme import Molecule, Reaction, KbEntry, DbXref

chebi_dir = '/home/fdrusso/work/data/chebi'

# Put all this in the `ref` db
refdb = kb.REFDB


# Load ChEBI data verbatim

In [2]:
compounds = {}
for row in pd.read_csv(os.path.join(chebi_dir, 'compounds.tsv'), sep='\t').itertuples():
    # Policy decision: use primary ID only; let the source worry about obsolete IDs.
    if row.STATUS == 'C' and pd.isnull(row.PARENT_ID):
        compounds[row.ID] = Molecule(_id=row.ID, name=row.NAME)
print(f'{len(compounds)} valid compounds')

59583 valid compounds


In [3]:
compound_names = collections.defaultdict(set)
for row in pd.read_csv(os.path.join(chebi_dir, 'names.tsv'), sep='\t').itertuples():
    if row.COMPOUND_ID in compounds:
        compound_names[row.COMPOUND_ID].add(row.NAME)
for compound_id, names in compound_names.items():
    compound = compounds[compound_id]
    compound.aka = list(names - {compound.name})

for row in pd.read_csv(os.path.join(chebi_dir, 'chemical_data.tsv'), sep='\t').itertuples():
    if row.COMPOUND_ID in compounds:
        if row.TYPE == 'MASS':
            compounds[row.COMPOUND_ID].mass = float(row.CHEMICAL_DATA)
        elif row.TYPE == 'CHARGE':
            compounds[row.COMPOUND_ID].charge = int(row.CHEMICAL_DATA)
        elif row.TYPE == 'FORMULA':
            compounds[row.COMPOUND_ID].formula = row.CHEMICAL_DATA

for row in pd.read_csv(os.path.join(chebi_dir, 'chebiId_inchi.tsv'), sep='\t').itertuples():
    if row.CHEBI_ID in compounds:
        compounds[row.CHEBI_ID].inchi = row.InChI

In [4]:
# Xrefs for select db's only
db_mapping = {
    'CAS Registry Number': 'CAS',
    'KEGG COMPOUND accession': 'KEGG',
    'KEGG GLYCAN accession': 'KEGG',
    'KEGG DRUG accession': 'KEGG',
    'MetaCyc accession': 'METACYC',
    'LINCS accession': 'LINCS',
    'Wikipedia accession': 'WIKI',
}

compound_xrefs = collections.defaultdict(set)
for row in pd.read_csv(os.path.join(chebi_dir, 'database_accession.tsv'), sep='\t').itertuples():
    if row.COMPOUND_ID in compounds and row.TYPE in db_mapping:
        compound_xrefs[row.COMPOUND_ID].add(DbXref(db_mapping[row.TYPE], row.ACCESSION_NUMBER))

for compound_id, xrefs in compound_xrefs.items():
        compounds[compound_id].xrefs = xrefs


In [5]:
compounds[17634]

Molecule [17634] D-glucose
  formula: C6H12O6
  mass: 180.15588 Da
  charge: +0

In [6]:
compounds[42758]

Molecule [42758] aldehydo-D-glucose
  formula: C6H12O6
  mass: 180.15588 Da
  charge: +0

# Put it in Mongo

In [7]:
kb.CODECS[Molecule].encode(compounds[42758])

{'_id': 42758,
 'name': 'aldehydo-D-glucose',
 'aka': ['Glucose',
  '(2R,3S,4R,5R)-2,3,4,5,6-pentahydroxyhexanal',
  'WURCS=2.0/1,1,0/[o2122h]/1/',
  'D-glucose',
  'Dextrose',
  'D(+)-Glucose',
  'aldehydo-D-gluco-hexose',
  'D-GLUCOSE IN LINEAR FORM'],
 'xrefs': [{'db': 'WIKI', 'id': 'Glucose'}, {'db': 'CAS', 'id': '50-99-7'}],
 'formula': 'C6H12O6',
 'mass': 180.15588,
 'charge': 0,
 'inchi': 'InChI=1S/C6H12O6/c7-1-3(9)5(11)6(12)4(10)2-8/h1,3-6,8-12H,2H2/t3-,4+,5+,6+/m0/s1'}

## Wipe and replace refdb.CHEBI

In [8]:
%%time
# Put this in db=ref, collection=chebi
refdb.CHEBI.drop()
for compound_id, compound in compounds.items():
    doc = kb.CODECS[Molecule].encode(compound)
    refdb.CHEBI.insert_one(doc)

CPU times: user 9.55 s, sys: 656 ms, total: 10.2 s
Wall time: 13.3 s


In [9]:
list(refdb.CHEBI.find({'aka': 'ribose'}).collation({'locale': 'en', 'strength': 1}))

[{'_id': 27476,
  'name': 'beta-D-ribopyranose',
  'aka': ['WURCS=2.0/1,1,0/[a222h-1b_1-5]/1/',
   'RIBOSE(PYRANOSE FORM)',
   'Ribose',
   'beta-D-Ribopyranose'],
  'xrefs': [{'db': 'KEGG', 'id': 'C08353'}, {'db': 'CAS', 'id': '7296-60-8'}],
  'formula': 'C5H10O5',
  'mass': 150.1299,
  'charge': 0,
  'inchi': 'InChI=1S/C5H10O5/c6-2-1-10-5(9)4(8)3(2)7/h2-9H,1H2/t2-,3-,4-,5-/m1/s1'},
 {'_id': 45506,
  'name': 'alpha-D-ribose',
  'aka': ['alpha-D-Rib',
   'RIBOSE',
   'alpha-D-ribofuranose',
   'WURCS=2.0/1,1,0/[a222h-1a_1-4]/1/'],
  'formula': 'C5H10O5',
  'mass': 150.1299,
  'charge': 0,
  'inchi': 'InChI=1S/C5H10O5/c6-1-2-3(7)4(8)5(9)10-2/h2-9H,1H2/t2-,3-,4-,5+/m1/s1'},
 {'_id': 47013,
  'name': 'D-ribofuranose',
  'aka': ['(3R,4S,5R)-5-(hydroxymethyl)tetrahydrofuran-2,3,4-triol',
   'ribose',
   'D-Ribose',
   'WURCS=2.0/1,1,0/[a222h-1x_1-4]/1/',
   'D-ribose'],
  'xrefs': [{'db': 'CAS', 'id': '613-83-2'},
   {'db': 'CAS', 'id': '50-69-1'},
   {'db': 'KEGG', 'id': 'C00121'}],
  'for

# Load RHEA master reactions verbatim

RHEA is organized around 'quartets'
- Master - indeterminate or unspecified direction
- irreversible left -> right
- irreversible right -> left
- explicitly reversible

Not clear what is gained by this representation vs say a reversibility attribute. One possibility is it's all about the cross-references to other reaction DBs. Need to explore a bit more.

## Main RHEA reaction definitions are in RDF

In [10]:
import rdflib
from rdflib.namespace import RDFS

RH = rdflib.namespace.Namespace('http://rdf.rhea-db.org/')
rhea_dir = '/home/fdrusso/work/data/rhea'

In [11]:
%time rhea_rdf = rdflib.Graph().parse(os.path.join(rhea_dir, 'rhea.rdf'))

rhea_rdf.bind('rh', RH)
rhea_rdf.bind('rdfs', RDFS)
rhea_rdf.bind('ch', rdflib.namespace.Namespace('http://purl.obolibrary.org/obo/'))
rhea_rdf.bind('ch2', rdflib.namespace.Namespace('http://purl.obolibrary.org/obo/chebi#'))
rhea_rdf.bind('ch3', rdflib.namespace.Namespace('http://purl.obolibrary.org/obo/chebi/'))

rhea_rdf.bind('UNIPROT', rdflib.namespace.Namespace('http://purl.uniprot.org/core/'))
rhea_rdf.bind('EC', rdflib.namespace.Namespace('http://purl.uniprot.org/enzyme/'))
rhea_rdf.bind('PUBMED', rdflib.namespace.Namespace('http://rdf.ncbi.nlm.nih.gov/pubmed/'))
rhea_rdf.bind('ECOCYC', rdflib.namespace.Namespace('http://identifiers.org/biocyc/ECOCYC:'))
rhea_rdf.bind('METACYC', rdflib.namespace.Namespace('http://identifiers.org/biocyc/METACYC:'))
rhea_rdf.bind('KEGG', rdflib.namespace.Namespace('http://identifiers.org/kegg.reaction/'))
rhea_rdf.bind('REACT', rdflib.namespace.Namespace('http://identifiers.org/reactome/'))
rhea_rdf.bind('MACIE', rdflib.namespace.Namespace('http://identifiers.org/macie/'))

CPU times: user 2min 43s, sys: 888 ms, total: 2min 44s
Wall time: 2min 43s


## Pull it into a more workable structure

In [12]:
scalars = {
    RH.id,
    RDFS.label,
    RH.equation,
    RH.status,
    RH.isTransport,
    
    RH.curatedOrder,
    
    RH.accession,
    RH.name,
    RH.formula,
    RH.charge,
    
    RH.location,
    RH.position,
    RH.polymerizationIndex,
    RH.underlyingChebi,
}
lists = {
    RH.ec,
    RDFS.seeAlso,
}
objects = {
    RH.bidirectionalReaction,
    RH.directionalReaction,

    RH.side,

    RH.contains1,
    RH.contains2,
    RH.contains3,
    RH.contains4,
    RH.contains5,
    RH.contains6,
    RH.contains7,
    RH.contains8,
    RH.contains9,
    RH.contains10,
    RH.contains11,
    RH.contains12,
    RH.contains13,
    RH.contains14,
    RH.contains15,
    RH.contains16,
    RH.contains17,
    RH.contains18,
    RH.contains19,
    RH.contains20,
    
    RH.contains21,
    RH.contains22,
    RH.contains24,
    RH.contains26,
    RH.contains27,
    RH.contains28,
    RH.contains32,
    RH.contains40,
    RH.containsN,
    RH.contains2n,
    RH.containsNplus1,
    RH.containsNminus1,

    RH.compound,
    RH.reactivePart,
}
drop = {
    RDFS.comment,
    RDFS.subClassOf,
    RH.citation,
    RH.chebi,
    RH.contains,
    RH.htmlEquation,
    RH.htmlName,
    RH.isChemicallyBalanced,
    RH.products,
    RH.substrates,
    RH.substratesOrProducts,
    RH.transformableTo,
}

def extract_value(g, o):
    if type(o) == rdflib.Literal:
        return o.toPython()
    else:
        return o.n3(g.namespace_manager)
    
def extract_object(g, s):
    result = {}
    for p, o in g[s]:
        if p in scalars:
            p = extract_value(g, p)
            o = extract_value(g, o)
            result[p] = o
        elif p in lists:
            p = extract_value(g, p)
            o = extract_value(g, o)
            if p in result:
                result[p].append(o)
            else:
                result[p] = [o]
        elif p in objects:
            p = extract_value(g, p)
            o = extract_object(g, o)
            if p in result:
                result[p].append(o)
            else:
                result[p] = [o]
        elif p not in drop:
            print(f'Ignoring {extract_value(g, s)} {extract_value(g, p)}')
    return result
    

## Coerce master reactions (only) into the KB Reaction structure

In [20]:
def to_dbxref(rhea_xref):
    # Special cases
    for prefix, db in [
        ('ch:GO_', 'GO'),
    ]:
        if rhea_xref.startswith(prefix):
            return DbXref(db, rhea_xref[len(prefix):])

    # Generally otherwise just split on a colon
    parts = rhea_xref.split(':')
    if len(parts) == 2:
        return DbXref(*parts)
    else:
        return DbXref('RHEA', rhea_xref)
    
    
def to_reaction(rhea_rxn):
    xrefs = set()
    if 'rh:ec' in rhea_rxn:
        xrefs.update(to_dbxref(rhea_xref) for rhea_xref in rhea_rxn['rh:ec'])
    if 'rdfs:seeAlso' in rhea_rxn:
        xrefs.update(to_dbxref(rhea_xref) for rhea_xref in rhea_rxn['rdfs:seeAlso'])
    # Lump in the rest of the quartet's xrefs -- not rigorous by RHEA standards, but reasonable for us.
    for key in ['rh:directionalReaction', 'rh:bidirectionalReaction']:
        for other_reaction in rhea_rxn.get(key, []):
            if 'rh:ec' in other_reaction:
                xrefs.update(to_dbxref(rhea_xref) for rhea_xref in other_reaction['rh:ec'])
            if 'rdfs:seeAlso' in other_reaction:
                xrefs.update(to_dbxref(rhea_xref) for rhea_xref in other_reaction['rdfs:seeAlso'])
    
    multipliers = [None, -1, +1]  # curatedOrder -> stoichiometry sign, 1-based
    stoichiometry = {}
    for side in rhea_rxn['rh:side']:
        multiplier = multipliers[side['rh:curatedOrder']]
        for count in range(1, 21):
            key = 'rh:contains' + str(count)
            for rhea_compound in side.get(key, []):
                chebi_id = rhea_compound['rh:compound'][0]['rh:accession']
                if chebi_id.startswith('CHEBI:'):
                    # We'll use the in-memory molecules for now, but this will utimately be a KB lookup
                    compound = compounds[int(chebi_id[6:])]
                    stoichiometry[compound] = multiplier * count
                else:
                    raise ValueError(f'Unrecognized compound ID {chebi_id}')
    
    reversible = False
    if 'rh:bidirectionalReaction' in rhea_rxn and rhea_rxn['rh:bidirectionalReaction'][0].get('rdfs:seeAlso'):
        reversible = True

    # RHEA does not provide nice names on its own; get the name from EC if possible
    ec_num = [xref for xref in xrefs if xref.db == 'EC']
    if ec_num:
        ec = kb.CODECS[KbEntry].decode(refdb.EC.find_one(ec_num[0].id))
        # Mark names as ambiguous where there are multiple ECs.
        name = ec.name if len(ec_num) == 1 else '* ' + ec.name
    else:
        # Fallback: use the RHEA-provided label, which is just the reaction formula.
        name = rhea_rxn['rdfs:label']

    return Reaction(
        _id = rhea_rxn['rh:id'],
        name = name,
        xrefs = xrefs or None,
        stoichiometry = stoichiometry,
        reversible = reversible,
    )

to_reaction(extract_object(rhea_rdf, RH['10164']))

Reaction(_id=10164, name='phosphopyruvate hydratase', shorthand=None, description=None, aka=None, xrefs={METACYC:2PGADEHYDRAT-RXN, ECOCYC:2PGADEHYDRAT-RXN, REACT:R-HSA-71660.2, REACT:R-HSA-70494.2, KEGG:R00658, GO:0004634, EC:4.2.1.11, MACIE:M0311}, stoichiometry={Molecule [58289] 2-phosphonato-D-glycerate(3-)
  formula: C3H4O7P
  mass: 183.0334 Da
  charge: -3: -1, Molecule [15377] water
  formula: H2O
  mass: 18.0153 Da
  charge: +0: 1, Molecule [58702] phosphonatoenolpyruvate
  formula: C3H2O6P
  mass: 165.0181 Da
  charge: -3: 1}, catalyst=None, reversible=True)

In [21]:
reactions = {}
skipped = {}
for s, _, _ in rhea_rdf.triples((None, RDFS.subClassOf, RH.Reaction)):
    rhea_rxn = extract_object(rhea_rdf, s)
    if rhea_rxn['rh:status'] == 'rh:Approved':
        try:
            reaction = to_reaction(rhea_rxn)
            reactions[reaction._id] = reaction
        
        except Exception as e:
            skipped[s] = (rhea_rxn, e)

print(f'{len(reactions)} reactions parsed succesfully, {len(skipped)} skipped')

10057 reactions parsed succesfully, 3830 skipped


In [22]:
kb.CODECS[Reaction].decode(kb.CODECS[Reaction].encode(reactions[11816]))

Reaction(_id=11816, name='glucose-6-phosphate isomerase', shorthand=None, description=None, aka=None, xrefs={GO:0004347, METACYC:PGLUCISOM-RXN, EC:5.3.1.9, ECOCYC:PGLUCISOM-RXN, KEGG:R00771}, stoichiometry={Molecule [57584] aldehydo-D-glucose 6-phosphate(2-): -1, Molecule [57579] D-fructose 6-phosphate(2-): 1}, catalyst=None, reversible=True)

## Wipe and replace refdb.RHEA

In [23]:
%%time
# Put this in db=ref, collection=rhea
refdb.RHEA.drop()
for reaction_id, reaction in reactions.items():
    doc = kb.CODECS[Reaction].encode(reaction)
    refdb.RHEA.insert_one(doc)

CPU times: user 2.1 s, sys: 135 ms, total: 2.23 s
Wall time: 2.85 s
